In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda() 
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda()  
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda()  
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda() 
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda()  
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda() 
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda()  
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda() 
    se1 = (torch.sum((model(xe1)-u_ex(xe1))**2)/xa1.size()[0]).cuda() 
    se2 = (torch.sum((model(xe2)-u_ex(xe2))**2)/xa1.size()[0]).cuda()  
    sf1 = (torch.sum((model(xf1)-u_ex(xf1))**2)/xa1.size()[0]).cuda()  
    sf2 = (torch.sum((model(xf2)-u_ex(xf2))**2)/xa1.size()[0]).cuda() 
    sg1 = (torch.sum((model(xg1)-u_ex(xg1))**2)/xa1.size()[0]).cuda()  
    sg2 = (torch.sum((model(xg2)-u_ex(xg2))**2)/xa1.size()[0]).cuda() 
    sh1 = (torch.sum((model(xh1)-u_ex(xh1))**2)/xa1.size()[0]).cuda()  
    sh2 = (torch.sum((model(xh2)-u_ex(xh2))**2)/xa1.size()[0]).cuda()
    
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2 + se1 + se2 + sf1 + sf2 + sg1 + sg2 + sh1 + sh2
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 16

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_8D_Dirichlet.npy", error_save)
np.save("DGM_relative_error_8D_Dirichlet.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(27031.7255, device='cuda:0')
current relative error is:  tensor(4.4353, device='cuda:0')
current epoch is:  50
current loss is:  tensor(3327.9464, device='cuda:0')
current relative error is:  tensor(1.2665, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1909.7493, device='cuda:0')
current relative error is:  tensor(0.9859, device='cuda:0')
current epoch is:  150
current loss is:  tensor(1214.1467, device='cuda:0')
current relative error is:  tensor(0.7630, device='cuda:0')
current epoch is:  200
current loss is:  tensor(709.0657, device='cuda:0')
current relative error is:  tensor(0.5625, device='cuda:0')
current epoch is:  250
current loss is:  tensor(424.6022, device='cuda:0')
current relative error is:  tensor(0.4252, device='cuda:0')
current epoch is:  300
current loss is:  tensor(271.1971, device='cuda:0')
current relative error is:  tensor(0.3164, device='cuda:0')
current epoch is:  350
current loss is:  tensor(190.3

current epoch is:  3050
current loss is:  tensor(31.6101, device='cuda:0')
current relative error is:  tensor(0.1357, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(31.0221, device='cuda:0')
current relative error is:  tensor(0.1349, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(31.5353, device='cuda:0')
current relative error is:  tensor(0.1313, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(29.8470, device='cuda:0')
current relative error is:  tensor(0.1332, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(31.0970, device='cuda:0')
current relative error is:  tensor(0.1305, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(30.2727, device='cuda:0')
current relative error is:  tensor(0.1294, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(29.4552, device='cuda:0')
current relative error is:  tensor(0.1294, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(29.852

current epoch is:  6100
current loss is:  tensor(13.7975, device='cuda:0')
current relative error is:  tensor(0.0881, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(13.4407, device='cuda:0')
current relative error is:  tensor(0.0896, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(13.5680, device='cuda:0')
current relative error is:  tensor(0.0923, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(12.3865, device='cuda:0')
current relative error is:  tensor(0.0843, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(12.5010, device='cuda:0')
current relative error is:  tensor(0.0888, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(12.2107, device='cuda:0')
current relative error is:  tensor(0.0841, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(12.4903, device='cuda:0')
current relative error is:  tensor(0.0881, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(11.869

current epoch is:  9200
current loss is:  tensor(3.9829, device='cuda:0')
current relative error is:  tensor(0.0513, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(4.1525, device='cuda:0')
current relative error is:  tensor(0.0484, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(3.8896, device='cuda:0')
current relative error is:  tensor(0.0441, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(4.0682, device='cuda:0')
current relative error is:  tensor(0.0483, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(3.6181, device='cuda:0')
current relative error is:  tensor(0.0453, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(4.3197, device='cuda:0')
current relative error is:  tensor(0.0476, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(3.7661, device='cuda:0')
current relative error is:  tensor(0.0478, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(3.8664, devic

current epoch is:  12250
current loss is:  tensor(2.4044, device='cuda:0')
current relative error is:  tensor(0.0343, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(1.9619, device='cuda:0')
current relative error is:  tensor(0.0338, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(2.5023, device='cuda:0')
current relative error is:  tensor(0.0382, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(2.0706, device='cuda:0')
current relative error is:  tensor(0.0327, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(2.0303, device='cuda:0')
current relative error is:  tensor(0.0321, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(2.0274, device='cuda:0')
current relative error is:  tensor(0.0356, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(2.0204, device='cuda:0')
current relative error is:  tensor(0.0305, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(2.142

...

current epoch is:  36650
current loss is:  tensor(0.6996, device='cuda:0')
current relative error is:  tensor(0.0229, device='cuda:0')
current epoch is:  36700
current loss is:  tensor(0.5652, device='cuda:0')
current relative error is:  tensor(0.0177, device='cuda:0')
current epoch is:  36750
current loss is:  tensor(0.6256, device='cuda:0')
current relative error is:  tensor(0.0174, device='cuda:0')
current epoch is:  36800
current loss is:  tensor(0.6371, device='cuda:0')
current relative error is:  tensor(0.0196, device='cuda:0')
current epoch is:  36850
current loss is:  tensor(0.5948, device='cuda:0')
current relative error is:  tensor(0.0184, device='cuda:0')
current epoch is:  36900
current loss is:  tensor(1.9746, device='cuda:0')
current relative error is:  tensor(0.0484, device='cuda:0')
current epoch is:  36950
current loss is:  tensor(0.5646, device='cuda:0')
current relative error is:  tensor(0.0166, device='cuda:0')
current epoch is:  37000
current loss is:  tensor(0.727

current epoch is:  39700
current loss is:  tensor(0.5132, device='cuda:0')
current relative error is:  tensor(0.0158, device='cuda:0')
current epoch is:  39750
current loss is:  tensor(0.5951, device='cuda:0')
current relative error is:  tensor(0.0186, device='cuda:0')
current epoch is:  39800
current loss is:  tensor(1.0431, device='cuda:0')
current relative error is:  tensor(0.0253, device='cuda:0')
current epoch is:  39850
current loss is:  tensor(0.9985, device='cuda:0')
current relative error is:  tensor(0.0248, device='cuda:0')
current epoch is:  39900
current loss is:  tensor(1.5126, device='cuda:0')
current relative error is:  tensor(0.0223, device='cuda:0')
current epoch is:  39950
current loss is:  tensor(0.7771, device='cuda:0')
current relative error is:  tensor(0.0218, device='cuda:0')
current epoch is:  40000
current loss is:  tensor(0.9809, device='cuda:0')
current relative error is:  tensor(0.0254, device='cuda:0')
current epoch is:  40050
current loss is:  tensor(1.142

current epoch is:  42750
current loss is:  tensor(1.4061, device='cuda:0')
current relative error is:  tensor(0.0301, device='cuda:0')
current epoch is:  42800
current loss is:  tensor(0.7661, device='cuda:0')
current relative error is:  tensor(0.0191, device='cuda:0')
current epoch is:  42850
current loss is:  tensor(1.1608, device='cuda:0')
current relative error is:  tensor(0.0367, device='cuda:0')
current epoch is:  42900
current loss is:  tensor(0.7340, device='cuda:0')
current relative error is:  tensor(0.0214, device='cuda:0')
current epoch is:  42950
current loss is:  tensor(0.5302, device='cuda:0')
current relative error is:  tensor(0.0171, device='cuda:0')
current epoch is:  43000
current loss is:  tensor(0.5925, device='cuda:0')
current relative error is:  tensor(0.0185, device='cuda:0')
current epoch is:  43050
current loss is:  tensor(0.5782, device='cuda:0')
current relative error is:  tensor(0.0214, device='cuda:0')
current epoch is:  43100
current loss is:  tensor(1.803

current epoch is:  45800
current loss is:  tensor(0.7167, device='cuda:0')
current relative error is:  tensor(0.0259, device='cuda:0')
current epoch is:  45850
current loss is:  tensor(0.6578, device='cuda:0')
current relative error is:  tensor(0.0191, device='cuda:0')
current epoch is:  45900
current loss is:  tensor(1.0273, device='cuda:0')
current relative error is:  tensor(0.0267, device='cuda:0')
current epoch is:  45950
current loss is:  tensor(0.7743, device='cuda:0')
current relative error is:  tensor(0.0217, device='cuda:0')
current epoch is:  46000
current loss is:  tensor(0.4828, device='cuda:0')
current relative error is:  tensor(0.0153, device='cuda:0')
current epoch is:  46050
current loss is:  tensor(0.6661, device='cuda:0')
current relative error is:  tensor(0.0218, device='cuda:0')
current epoch is:  46100
current loss is:  tensor(0.5180, device='cuda:0')
current relative error is:  tensor(0.0152, device='cuda:0')
current epoch is:  46150
current loss is:  tensor(0.495

current epoch is:  48850
current loss is:  tensor(1.2209, device='cuda:0')
current relative error is:  tensor(0.0387, device='cuda:0')
current epoch is:  48900
current loss is:  tensor(0.4627, device='cuda:0')
current relative error is:  tensor(0.0154, device='cuda:0')
current epoch is:  48950
current loss is:  tensor(0.4377, device='cuda:0')
current relative error is:  tensor(0.0150, device='cuda:0')
current epoch is:  49000
current loss is:  tensor(0.4598, device='cuda:0')
current relative error is:  tensor(0.0141, device='cuda:0')
current epoch is:  49050
current loss is:  tensor(0.4729, device='cuda:0')
current relative error is:  tensor(0.0176, device='cuda:0')
current epoch is:  49100
current loss is:  tensor(0.4322, device='cuda:0')
current relative error is:  tensor(0.0150, device='cuda:0')
current epoch is:  49150
current loss is:  tensor(0.6978, device='cuda:0')
current relative error is:  tensor(0.0236, device='cuda:0')
current epoch is:  49200
current loss is:  tensor(0.519

In [18]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.0137],
        [0.0327],
        [0.0028],
        ...,
        [0.0147],
        [0.0421],
        [0.0763]], device='cuda:0', grad_fn=<AbsBackward>)